# Генерация игровых диалогов с помощью больших языковых моделей

Диалог — один из ключевых инструментов повествования в видеоиграх. Через него игрок узнаёт мир, персонажей и последствия собственных решений.

В этом ноутбуке мы исследуем, как большие языковые модели могут использоваться для генерации игровых диалогов между главным героем и NPC. Мы рассмотрим, каким образом можно задавать контекст сцены, описывать персонажей и получать на выходе связные, вариативные и управляемые диалоги, пригодные для использования в игровом прототипе.

## Постановка задачи
В данном ноутбуке будет решаться задача генерации дерева диалога главного героя с NPC.

Важно, что наша команда отвечает только за логику генерации диалогов, поэтому входные параметры и формат выхода зафиксирован договорённостями с другими командами. 

Ограничения:
- Задачу нужно решить с помощью одного запроса в LLM
- В качестве входных параметров персонажей и игрового окружения должен быть использован набор параметров, указанный в [файле](models.py).
- В результате работы сервиса генерации должен получиться JSON со следующей структурой:
```python
{
    "data": [
        {
            "id": id_вершины,
            "info": "Краткое содержание текста в вершине",
            "line": "Текст в вершине",
            "to": [
                {
                    "id": id_вершины_в_котороую_ведёт_ребро,
                    "info": "Краткое содержание текста в ребре",
                    "line": "Текст в ребре"
                }
            ] # список вершин, куда идут рёбра
        }
    ]
}
```

In [1]:
from dataclasses import dataclass

@dataclass(frozen=True)
class GenerationPrompts:
    structure: str
    node_content: str
    edge_content: str

In [2]:
PROMPT_STRUCTURE = '''
Создай структуру диалогов для компьютерной игры в виде списка смежности ориентированного ациклического графа. Учитывай следующие требования:
<Характеристики структуры>
	<Структура тип=вершина>
		<Типы вершин>
			- Тип C (Choice Nodes) - вершины, в которых у игрока есть от $mn_answers_cnt до $mx_answers_cnt вариантов ответа. Ответы в C-вершинах **должны** влиять на сюжет/отношения 
			- Тип M (Monologue Nodes) - вершины, где NPC раскрывает характер/историю через монолог. Игрок **только слушает** (**единственный** вариант ответа: "Продолжить"). Каждый M-узел имеет выходную степень 1. Лимит: **не более 2 M-узлов подряд** в любой ветке. Обязательно **проверь** что в итоговом графе **нет более 2 M-узлов подряд**.
			- Тип P (Pendant Nodes) - вершины, в которых заканчивается диалог. Каждая P-вершина имеет выходную степень 0. Число реплик NPC до каждого P-узла должно лежать в диапазоне от $mn_depth до $mx_depth.
		</Типы вершин>
		<Формат>
			$json_node_structure
		</Формат>
		<Инструкции>
			- id: Сгенерируй уникальный числовой идентификатор для текущей вершины
			- info: Напиши тематику монолога NPC в косвенной речи. Тематику сформулируй в формате одного-двух предложений
			- type: Определи тип данной вершины
			- mood: Выбери настроение NPC при произнесении монолога в данной вершине из следующего списка: $moods_list,
			- goal_achieve: Информация о том, чего достиг игрок в данной вершине.
				- item: Опиши достижение игроком цели на получение предмета в данной вершине В данном поле должен храниться только id предмета, который получил игрок или -1 в случае, когда игрок не получал никакого предмета. Сопоставь полученный предмет с его id в соответствии со следующим словарём: $items_dict. Если предмета нет в словаре, поставь в данное поле 0.
				- info: Опиши достижение игроком цели на получение информации в данной вершине. В данном поле должна храниться только строка - информация полученная игроком в данной вершине или -1 в случае, когда игрок не получал никакой информации.
			- to: Создай список вершин для перехода в зависимости от ответов игрока
				- id: Сгенерируй уникальный числовой идентификатор для следующей вершины
				- mood: Выбери настроение игрока при произнесении реплики в данном ребре из следующего списка: $moods_list
		</Инструкции>
	</Структура тип=вершина>
	<Структура тип=граф>
		<Формат>
			$json_structure
		</Формат>
	</Структура тип=граф>
</Характеристики структуры> 
<Характеристики диалога>
	<Характеристика тип=NPC>
		<Имя>$NPC_name</Имя>
		<Стиль речи>$NPC_talk_style</Стиль речи>
		<Профессия>$NPC_profession</Профессия>
		<Внешний вид>$NPC_look</Внешний вид>
		<Взаимоотношения с игроком>Отношение NPC к игроку - $NPC_to_hero_relation</Взаимоотношения с игроком>
		<Черты характера>$NPC_traits</Черты характера>
		<Дополнительная информация>$NPC_extra</Дополнительная информация>
	</Характеристика тип=NPC>
	<Характеристика тип=игрок>
		<Имя>$hero_name</Имя>
		<Стиль речи>$hero_talk_style</Стиль речи>
		<Профессия>$hero_profession</Профессия>
		<Внешний вид>$hero_look</Внешний вид>
		<Взаимоотношения с NPC>Отношение игрока к NPC - $hero_to_NPC_relation</Взаимоотношения с NPC>
		<Черты характера>$hero_traits</Черты характера>
		<Дополнительная информация>$hero_extra</Дополнительная информация>
	</Характеристика тип=игрок>	
	<Характеристика тип=окружение> 
		**Обязательно** учитывай характеристики окружения, в котором происходят события диалога: $scene
	</Характеристика тип=окружение>
	<Характеристика тип=игровой мир>
		<Жанр>$genre</Жанр>
		<Исторический период>$epoch</Исторический период>
		<Тональность>$tonality</Тональность>
		<Описание>$world_settings</Описание>
	</Характеристика тип=игровой мир>
	<Цели>
		Ты **обязан** сделать так, чтобы в конце диалога каждая из следующих целей была достигнута **хотя бы в одной** из P-вершин: $goals
		<Формат>	
			<Тип> Тип события в конце диалога </Тип>
			<Объект> Объект участвующий в происходящем событии </Объект>
			<Условие> Условие, которое **должен** выполнить игрок для достижения цели </Условие>
		</Формат>		
	</Цели>
	<Контекст диалога>В этом поле содержится описание краткой предыстории и ключевых событий, происходящих в диалоге. **Учитывай** их при генерации структуры: $context</Контекст диалога>
	<Дополнительная информация>Дополнительная информация о диалоге: $extra</Дополнительная информация>
</Характеристики диалога>
<Инструкции>
	<Инструкции тип=диалог>
		При генерации тематик диалога **строго** соблюдай следующие инструкции:
		- Твоя задача прописать в информации о вершинах **только тематики реплик**.
		- Ты **обязан** быть естественным и учитывать, что NPC и игрок могут менять тему, но без резких скачков.
		- Некоторые линии диалогов **могут** не привести к достижению поставленной цели.
	</Инструкции тип=диалог>
	<Инструкции тип=структура>
		При генерации структуры графа **строго** соблюдай следующие инструкции:
		- Ты **обязан** соблюдать все требования по структурам данных и учитывать все характеристики
		- P-вершины **должны** завершать диалог логично
		- **Обязательно проверь**, что **нет** рёбер, ведущих в несуществующие вершины.
		- Полученный граф **должен** быть связным
	</Инструкции тип=структура>
	В ответ верни **только JSON** без пояснений, сохраняя все поля из примера
</Инструкции>	
Строго соблюдай все требования. Сделай структуру диалога логичной и интересной. **Перепроверь**, выполнил ли ты все требования и инструкции.
Теперь создай структуру диалога:

'''.strip()

PROMPT_NODE_CONTENT = '''
Твоя задача - создать монолог, который будет являться логичным продолжением каждой из следующих цепочек диалога от лица NPC:
<Цепочки диалога>
	$chain
</Цепочки диалога>
<Характеристики>
	При генерации **обязательно** учитывай следующие характеристики: 
	<Тематика> Ты **обязан** сделать так, чтобы реплика подходила под следующую тематику: $tematic </Тематика>
	<Характеристика тип=NPC>
		Обязательно учитывай характеристики NPC, **особенно** отношение NPC к игроку. Пиши только текст и не описывай действия.  
		<Имя>$name</Имя>
		<Стиль речи>$talk_style</Стиль речи>
		<Профессия>$profession</Профессия>
		<Внешний вид>$look</Внешний вид>
		<Взаимоотношения NPC с игроком>Отношение NPC к игроку - $relation</Взаимоотношения NPC с игроком>
		<Черты характера>$traits</Черты характера>
		<Настроение>Ты **обязан** учитывать, что настроение NPC - **$mood** при произнесении монолога</Настроение>
		<Внешний вид>$look</Внешний вид>
		<Дополнительная информация>$NPC_extra</Дополнительная информация>
	</Характеристика тип=NPC>
	<Характеристика тип=окружение> 
		Обязательно учитывай характеристики окружения, в котором происходят события диалога: $scene
	</Характеристика тип=окружение>
	<Характеристика тип=игровой мир>
		$world_settings
	</Характеристика тип=игровой мир>
	<Дополнительная информация>$extra</Дополнительная информация>
</Характеристики>
<Инструкции>
	- Ты **обязан** соблюдать все инструкции и учитывать все характеристики NPC
	- Ты **должен** сделать так, чтобы сгенерированные реплики не повторялись
	- Монолог **должен** логично продолжать диалог, без резких смен темы и скачков.
	- В ответ ты **обязан** вернуть только монолог - текст, без лишних знаков и дополнительных пояснений
</Инструкции>
Строго соблюдай все требования. Сделай монолог логичным и интересным. **Перепроверь**, выполнил ли ты все требования и инструкции.
Теперь создай монолог:
'''.strip()


PROMPT_EDGE_CONTENT = '''
Твоя задача - создать набор из $replic_cnt реплик игрока, которые будут служить **триггерами** для перехода NPC к определенным тематикам в ответе:
<Тематики> 
	<Формат> 
		$json_tematics
	</Формат> 
	<Описание>
		- id: Уникальный числовой идентификатор для данной тематики
		- info: Содержание тематики
		- mood: Настроение, с которым игрок произносит реплику для перехода NPC к данной тематике
	</Описание>
	Ты **обязан** сделать так, чтобы при использовании реплики игроком, NPC мог перейти к своей реплике с одной из следующих тематик: $tematics. Каждая тематика должна быть задействована **ровно 1 раз**
</Тематики>
Все созданные тематики **должны** являться **логичным продолжением** каждой из следующих цепочек диалога:
<Цепочки диалога>
	$chain
</Цепочки диалога> 
<Структура>
	<Формат>
		В ответ ты **обязан** вернуть только список реплик **в следующем формате JSON**:
		$json_edge_structure
	</Формат>
	<Инструкции>
		- id: напиши id тематики, к которой перейдёт NPC после данной реплики,
		- line: создай реплику - текст в формате 1-3 предложений, **без лишних знаков** и дополнительных пояснений.
		- info: создай краткое описание реплики в косвенной речи, **2-3 слова без лишних знаков** и дополнительных пояснений. **Перепроверь**, соответствует ли описание репликеЫ.
	</Инструкции>
</Структура>
<Характеристики>
	При генерации **обязательно** учитывай следующие характеристики: 
	<Следующие тематики> Ты **обязан** сделать так, чтобы при произнесении какой-то из созданных реплик, монолог NPC **должен** перейти к одной из следующих тематик: $tematics. Каждая тематика должна быть задействована ровно 1 раз. Порядок вывода созданных реплик **должен** соответствовать порядку тематик.</Следующие тематики>
	<Характеристика тип=игрок>
		Обязательно учитывай характеристики игрока, **особенно** отношения игрока к NPC. Пиши только текст и не описывай действия.  
		<Имя>$name</Имя>
		<Стиль речи>$talk_style</Стиль речи>
		<Профессия>$profession</Профессия>
		<Внешний вид>$look</Внешний вид>
		<Взаимоотношения с NPC>Отношение игрока к NPC - $relation</Взаимоотношения с NPC>
		<Черты характера>$traits</Черты характера>
		<Внешний вид>$look</Внешний вид>
		<Дополнительная информация>$hero_extra</Дополнительная информация>
	</Характеристика тип=игрок>
	<Характеристика тип=окружение> 
		Обязательно учитывай характеристики окружения, в котором происходят события диалога: $scene
	</Характеристика тип=окружение>
	<Характеристика тип=игровой мир>
		$world_settings
	</Характеристика тип=игровой мир>
</Характеристики>
<Инструкции>
	- Ты **обязан** соблюдать все инструкции и учитывать все характеристики игрока
	- Ты **должен** сделать так, чтобы сгенерированные реплики не повторялись
	- **Каждая** реплика должна **логично** продолжать диалог, без резких смен темы и скачков. 
</Инструкции>
Строго соблюдай все требования. Сделай реплику логичной и интересной. **Перепроверь**, выполнил ли ты все требования и инструкции.
Теперь создай реплику:
'''

In [3]:
from models import CharacterParameters, DialogParamters, GameParameters


game = GameParameters(
    world_settings=(
        "Алматы недалёкого будущего: город-полигон, где университеты и корпорации "
        "тестируют роботов-курьеров. Над городом — сеть дронов-наблюдателей, "
        "в подвалах — подпольные мастерские."
    ),
    genre="приключения",
    epoch="будущее (2050-е)",
    tonality="нейтральная с элементами юмора",
)

dialog = DialogParamters(
    scene=(
        "Ночной двор возле старого общежития. Снег скрипит, фонари мигают, "
        "рядом стоит робот-курьер с помятым коробом."
    ),
    context="Герой пытается получить доступ в здание, а NPC решает, помогать ему или нет.",
    mn_depth=3,
    mx_depth=5,
    mn_answers_cnt=2,
    mx_answers_cnt=4,
    NPC_to_hero_relation="настороженно-доброжелательное",
    hero_to_NPC_relation="вежливое, но настойчивое",
    extra="Диалог должен быть понятен школьникам, без грубости и сленга 18+.",
)

npc = CharacterParameters(
    name="Айбар",
    profession="вахтёр",
    talk_style="коротко, по делу, иногда ворчит",
    traits="подозрительный, но справедливый; любит порядок",
    look="в тёплой куртке, с термосом и связкой ключей",
    extra="постоянно проверяет бейджи и документы",
)

hero = CharacterParameters(
    name="Мира",
    profession="студентка-робототехник",
    talk_style="спокойно, аргументирует, задаёт уточняющие вопросы",
    traits="любознательная, упорная, не любит конфликты",
    look="в шапке и с рюкзаком, на рукаве нашивка кружка робототехники",
    extra="спешит, потому что у неё важная встреча",
)

In [4]:
params = {
    "world_settings": game.world_settings,
    "genre": game.genre,
    "epoch": game.epoch,
    "tonality": game.tonality,

    "scene": dialog.scene,
    "context": dialog.context,
    "mn_depth": dialog.mn_depth,
    "mx_depth": dialog.mx_depth,
    "mn_answers_cnt": dialog.mn_answers_cnt,
    "mx_answers_cnt": dialog.mx_answers_cnt,
    "NPC_to_hero_relation": dialog.NPC_to_hero_relation,
    "hero_to_NPC_relation": dialog.hero_to_NPC_relation,
    "extra": dialog.extra,

    "npc": npc.model_dump(),
    "hero": hero.model_dump(),
}

In [5]:
prompts = GenerationPrompts(
    structure=PROMPT_STRUCTURE,
    node_content=PROMPT_NODE_CONTENT,
    edge_content=PROMPT_EDGE_CONTENT,
)

In [ ]:
import os

from openai import OpenAI, base_url

from generator import DialogGenerator
from settings import LLMSettings

llm_settings = LLMSettings()

client = OpenAI(
    api_key="sk-b4d8c96a0b984582ad8ecf6fe...",
    base_url="https://api.deepseek.com"
)

gen = DialogGenerator(
    client,
    model_structure="deepseek-chat",
    model_dialogue="deepseek-chat",
    system_prompt=llm_settings.get_system_prompt(),
    prompts=prompts,
    llm_settings=llm_settings,
    params=params,
)

dialog_json = gen.run()


Создай структуру диалогов для компьютерной игры в виде списка смежности ориентированного ациклического графа. Учитывай следующие требования:
<Характеристики структуры>
	<Структура тип=вершина>
		<Типы вершин>
			- Тип C (Choice Nodes) - вершины, в которых у игрока есть от 2 до 4 вариантов ответа. Ответы в C-вершинах **должны** влиять на сюжет/отношения 
			- Тип M (Monologue Nodes) - вершины, где NPC раскрывает характер/историю через монолог. Игрок **только слушает** (**единственный** вариант ответа: "Продолжить"). Каждый M-узел имеет выходную степень 1. Лимит: **не более 2 M-узлов подряд** в любой ветке. Обязательно **проверь** что в итоговом графе **нет более 2 M-узлов подряд**.
			- Тип P (Pendant Nodes) - вершины, в которых заканчивается диалог. Каждая P-вершина имеет выходную степень 0. Число реплик NPC до каждого P-узла должно лежать в диапазоне от 3 до 5.
		</Типы вершин>
		<Формат>
			
        {
        "id": "*Уникальный числовой идентификатор для текущей вершины*", 
        "

{'id': 1,
 'info': 'Айбар, вахтёр, проверяет документы Миры у входа в общежитие. Он подозрительно осматривает её и робота-курьера.',
 'type': 'C',
 'mood': 'нейтральный',
 'goal_achieved': {'item': -1, 'info': -1},
 'line': 'Пропуск в порядке, но этот робот... У него маркировка курьерской службы "Экспресс-Доставка", а не университетского технопарка. По регламенту, сторонние автоматизированные единицы без специального пропуска в жилой корпус не допускаются. Вы можете оставить посылку здесь, на посту, и забрать её лично после предъявления студенческого билета. Или дождаться, пока робот получит временный пропуск в отделе логистики. Это правила. Безопасность проживающих — моя обязанность.',
 'to': [{'mood': 'воодушевлёный',
   'line': 'Ты так спешишь, у тебя что-то важное? Может, покажешь пропуск?',
   'info': 'интересуется причиной спешки',
   'id': 2},
  {'mood': 'нейтральный',
   'line': 'Смотри, робот-курьер с помятым коробом стоит. Может, ему помочь?',
   'info': 'предлагает помощь ро